# 陣列運算

上一堂課的回測，大家是否都注意到雖然我們的結果是正確的，程式執行的時間卻很慢呢？

針對不同量級的資料，我們需要用不同的做法去解決問題，資料越大，我們就需要用聰明的方法做事

要加快程式的做法有很多種，像是用時間複雜度較好的演算法

但是演算法在實作上**十分困難**，而且若處理的資料量太小，很難看出時間上的差異

另一種做法是懂得利用資料結構，像是**陣列**(或者以 Python 的説法：List)

In [ ]:
import xlwings as xw
import time

In [ ]:
# 以我們上一堂課的 Excel 檔案爲例，打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r'你的 tsmc_back_test.xlsx 路徑')
wb

In [ ]:
tsmc_sheet = wb.sheets['2330']

result_sheet = wb.sheets['result']

In [ ]:
# 我們可以算出所有的日報酬率
from xlwings.constants import Direction

num_of_rows = tsmc_sheet.range('B1').end(Direction.xlDown).row
num_of_rows

# 記錄時間，time.time() 就像是按碼錶，把執行到它的那一刻的時間記錄下來
ts = time.time()

for i in range(3, num_of_rows+1):
    daily_return = (result_sheet.cells(i, 2).value - result_sheet.cells(i-1, 2).value) / result_sheet.cells(i-1, 2).value
    result_sheet.cells(i, 3).value = daily_return

# 把回圈的跑完的時間記錄下來
td = time.time()
td - ts

# 其實這個速度很慢...
- 原因是讀寫 Excel 試算表内的資料是很費時的
- 若想加速，需要**減少讀寫 Excel 資料的次數**
- 一個簡單的原則：**當程式碼出現一次 .value, 我們的程式就會需要讀寫一次 Excel
- 所以目前的程式碼是讀寫 4 x N 次

# 這時我們就可以使用 Python 的 List (清單) 來解決執行效能的問題

- List (清單) 其實就是一般程式語言的 Array (陣列) 只是用了不同的名字 
- 可以一次性的截取出所有的收盤價，存入 list，這樣當 Python 在進行運算時
- 可以到陣列讀取值，而非到 Excel 試算表讀取 Excel 的資料

## list (清單)

arr = [224.5, 224.5, 233.0, 237.5, 237.5, 238.0]

| arr   |  224.5 |224.5  | 233.0   | 237.5   | 237.5   | 238.0 |
| ------ | --- | --- | --- | --- | --- | --- |
| 索引值 | 0   | 1   | 2   | 3   | 4   | 5   |

```python
arr[0]
# 會回傳 224.5
arr[5]
# 會回傳 237.5
```

---
## .append()

把資料加到清單的最後面：

```
ary = [1,2,3]
ary.append(4)
```
最後產生的 list 就會是：
```python
[1, 2, 3, 4]
```

---

## slicing (切片)

Slicing 就是取出一個陣列部分裏面的單個或多個值

實作方法是，**ary[起始的索引值 : 結束的索引值+1]**

```python
ary[0:3]
```
0 代表起始的索引值、3代表結束的索引值的後面一個值

最後就會得到 
```python
[1,2,3]
```

In [ ]:
ary = [1, 2, 3, 4]
print(ary[0])
ary.append(5)
print(ary)
ary[0:3]

In [ ]:
# expand='down' 表示會自動從 B2 開始，往下搜尋到最後一個有值的儲存格，以陣列的形式回傳
tsmc_price = tsmc_sheet.range('B2').options(expand='down').value
tsmc_price

In [ ]:
# 先宣告一個空陣列
return_values = []

# 從第二筆資料(索引值為1)開始
for i in range(1, len(tsmc_price)):
    # 取得當天的收盤價
    today_price = tsmc_price[i]
    # 取得前一天的收盤價
    yesterday_price = tsmc_price[i-1]
    # 算出報酬率
    return_value = (today_price - yesterday_price) / yesterday_price
    # 將報酬率存入陣列
    return_values.append(return_value)
    
return_values

In [ ]:
# 接下來我們就可以歡天喜地的把陣列賦值給 C3 到 C114 這個範圍了

result_sheet.range("C3").value = return_values

# 但是結果有點出乎我們的意料...

![](https://www.dropbox.com/s/tgsno7c6d6jiyaq/range_wrong.PNG?dl=1)

- 原因是陣列是以 row-based，但是我們想要賦值的範圍是 column-based

若你執行以下程式碼：
```python
result_sheet.range("A1:B2").value
```
就會回傳
```python
[['日期', '收盤價'], [datetime.datetime(2017, 10, 5, 0, 0), 224.5]]
```
在 Excel 的世界，任何一個二維 range 的值，就是用一個**二維陣列(也就是在陣列裏放另一個陣列)來表示**

- 而目前 return_values 這個陣列的大小是 1 x 111 
- 我們需要有一個方法把 return_values 的大小轉換成 111 x 1

In [ ]:
result_sheet.range("A1:B2").value

In [ ]:
# 我們需要把 sma 的每一個值分別放入一個 list，一個 list 代表一個 row 的值
# 這些代表 row 的 list 最後會被放入另一個 list, 成爲一個二維陣列
values2d = [[227.33333333],
            [231.66666667],
            [236.        ]]

result_sheet.range("C3").value = values2d

# 在這裏就和各位介紹 Numpy 

這時就介紹一下 Numpy，它是 Python 的一個重要模組，主要用於資料處理上。Numpy 底層以 C 和 Fortran 語言實作，所以能快速操作多重維度的陣列。

因爲 Numpy 這個殺手級模組的出現，讓 Python 語言在資料科學與科研的領域大受歡迎。

- 在這個範例我們就來看如何利用 **Numpy** 這個熱門的科學運算套件所提供的資料結構

In [ ]:
# 引用 numpy
import numpy as np

## numpy.reshape()

改變一個陣列的維度

假設原本我們有一個 1 x 6 的陣列：

```python
[0, 1, 2, 3, 4, 5]
```

我們將該陣列轉成一個 **numpy** 的陣列

```python
np.array(ary)
```

array([0, 1, 2, 3, 4, 5])

再用 **reshape** 把它變成一個 3 x 2 的陣列

```python
np.array(ary).reshape(3,2)
```

最後的陣列就會變成：

```python
array([[0, 1],
       [2, 3],
       [4, 5]])
```

In [ ]:
ary = [0, 1, 2, 3, 4, 5]
np.array(ary)

In [ ]:
np.array(ary).reshape(6,1)

In [ ]:
# expand='down' 表示會自動從 B2 開始，往下搜尋到最後一個有值的儲存格，以陣列的形式回傳
tsmc_price = tsmc_sheet.range('B2').options(expand='down').value
tsmc_price

In [ ]:
# 引用 numpy
import numpy as np

# 先將 return_values 轉換成 numpy array
return_array = np.array(return_values)
return_array

In [ ]:
array_length = len(return_array)
array_length

In [ ]:
# 接下來我們就用 reshape(row, column) 改變陣列大小
return_array = return_array.reshape(array_length, 1)
return_array

In [ ]:
"""
現在當你需要把所有的報酬率寫入 C欄，你的做法可能會是：
result_sheet.range("C3:C114").value = return_array
最後在針對 range 賦值時，xlwings 只需指定起點，也就是最上面的 cell
若你的 range 是二維陣列，就是左上角的 cell
"""
result_sheet.range("C3").value = return_array

In [ ]:
# 最後我們來測試陣列運算所花費的時間
ts_array = time.time()

return_values = []

for i in range(1, len(tsmc_price)):
    today_price = tsmc_price[i]
    yesterday_price = tsmc_price[i-1]
    return_value = (today_price - yesterday_price)/yesterday_price
    return_values.append(return_value)

result_sheet.range("C3").value = return_array

td_array = time.time() 

td_array - ts_array

In [ ]:
print("時間差距：{}倍".format((td - ts)/(td_array - ts_array)))

In [ ]:
# 接下來我們就來計算3日移動平均

# 先宣告一個空陣列
sma = []

# 從陣列的第三個值開始計算
for i in range(3, len(tsmc_price)+1):
    # 一次取出三天的收盤價，算出平均值，再用 append 加入陣列
    mean = sum(tsmc_price[i-3:i]) / len(tsmc_price[i-3:i])
    print(mean)
    
# 當然，這個做法有點土炮，畢竟 numpy 已經提供了很好用的功能可以用來計算平均值

## numpy.mean()

計算一個 np array 的平均值
```python
ary = [1, 2, 3]
np_array = np.array(ary)

np.mean(np_array)
```
就會把 np_array 的平均值，也就是 2.0 算出來

In [ ]:
sma = []
# 從陣列的第三個值開始計算
for i in range(3, len(tsmc_price)+1):
    # 一次取出三天的收盤價，算出平均值，再用 append 加入陣列
    mean = sum(tsmc_price[i-3:i]) / len(tsmc_price[i-3:i])
    sma.append(mean)

# 最後再把我們的結果轉成 N x 1 大小的陣列
sma_array = np.array(sma).reshape(len(sma),1)
# 最後寫回 D 欄
result_sheet = wb.sheets['result']
result_sheet.range('D4').value = sma_array

# range().options() 功能

除了能夠用 expand 自動偵測範圍以外，options 也能讓開發者指定資料的型別

```python
# 從 B2 往下查找，回傳一個 np.array 陣列
tsmc_sheet.range('B2').options(np.array, expand='down')
```

In [ ]:
# expand='down' 表示會自動從 B2 開始，往下搜尋到最後一個有值的儲存格，以 np.array的形式回傳
tsmc_price = tsmc_sheet.range('B2').options(np.array, expand='down').value
tsmc_price

# numpy 陣列相乘

傳統上若我們要用陣列做 element-wise 的運算...

```python
km_list = [3, 5, 10, 21, 42.195]
km_to_mile = 0.621371192
mile_list = []
# 一定需要一個 for loop 
for km in km_list:
    mile_list.append(km * km_to_mile)

print(mile_list)
```

用 lambda 函數也沒有好到那裏去...

```python
km_list = [3, 5, 10, 21, 42.195]
mile_list = list(map(lambda x: x * 0.621371192, km_list))
print(mile_list)
```

但是今天你若是使用 numpy array...

```python
km_list = [3, 5, 10, 21, 42.195]
km_array = numpy.array(km_list)
print(type(km_array))
km_to_mile = 0.621371192
# 讓一個 numpy 裏面的每一筆資料都與常數 km_to_mile 相乘
# 使用 numpy, 語法非常直覺！
mile_array = km_array * km_to_mile
print(mile_array)
```


# 隨堂練習
- 請用 numpy 練習計算這五個人的 BMI
- 回傳的多筆 bmi 值必須是一個陣列
```python
 heights = [173, 168, 171, 189, 179]
weights = [65.4, 59.2, 63.6, 88.4, 68.7]
```

---

In [ ]:
# 接下來我們就來計算 5日加權移動平均

# 需要將權重與價格兩兩相乘
weights = np.arange(1, 6)
print(weights)

prices = tsmc_price[0:5]
print(prices)

# 使用 weight * prices, 將陣列的數依照順序兩兩相乘, 非常强大！
weights * prices

# 隨堂練習

```python
# 算出所有的 5日加權移動平均,寫入 E 欄
wma = []

weights = list(range(1,6))

for i in range(5, len(tsmc_price)+1):
    prices = ______________
    
    result = _________________________
    wma.append(result)

wma_array = np.array(wma).reshape(len(wma), 1)
result_sheet.range((6, 'E')).value = wma_array
```

# 小結
- 陣列運算相對不直覺，但是**運算效能卻會大幅提升**
- numpy 這個强大的套件就在功能和效能上補足了原廠 python list 的不足之處

補充教材：

[numpy 套件官網](http://www.numpy.org/)

# 資料視覺化

今天我們手上已經有了許多資料，但是受到生活的限制，很多時候我們對數字是無感的

因此，把資料視覺化，像是產生圖表，就成了一個重要并且值得研究的議題

In [ ]:
# xlwings 直接整合了 Excel 原廠的 Chart 物件 
chart = result_sheet.charts.add()
# 使用 expand 將試算表内所有、連續的資料撈出
chart.set_source_data(result_sheet.range('A1').expand())
chart.chart_type = 'line'
chart.top = result_sheet.range('K5').top

# Matplotlib

Python 開發者社群最被廣汎使用的繪圖套件，功能强大

In [ ]:
import matplotlib.pyplot as plt

plt.ylabel('some numbers')
plt.plot([1, 2, 3, 4], [1, 4, 9, 16])
plt.show()

In [ ]:
tsmc_sheet = wb.sheets['result']
time_series = tsmc_sheet.range('A2').options(np.array, expand='down').value
time_series

In [ ]:
ma_3d = tsmc_sheet.range('D2').options(np.array, expand='down').value
ma_3d

In [ ]:
fig = plt.figure()
plt.plot(time_series, tsmc_price)
plt.xlabel('Date')
plt.ylabel('Price')

In [ ]:
result_sheet.pictures.add(fig, name="Closing Price vs Date")

# 成功將圖加到了 Excel 上面

![](https://www.dropbox.com/s/hkr2r8r8vn98msj/matplot_first_try.PNG?dl=1)

但是圖的位置將一部分的資料蓋住了...

In [ ]:
# 設定圖的左邊與上面的位置
plot = result_sheet.pictures.add(fig,name="Closing Price vs Date",
                     left=result_sheet.range('F10').left, top=result_sheet.range('F10').top)

In [ ]:
# 若覺得圖太大...

plot.height /= 2
plot.width /= 2